In [2]:
###############---------------------Pipeline de refinamiento-------------------####################
#1.Carga de Datos 
#2.Clusterización de clientes
#3️.Preparacion y creacion de proxys

import pandas as pd
import numpy as np
import os
import joblib
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

class ClusterPipeline:
    """
    Pipeline para la transformación refinada de datos antes del modelado.
    Incluye carga de datos, clusterización de clientes y asociación para entrenamiento.
    """
    
    def __init__(self, file_path, output_dir, num_clusters=5, random_state=42):
        """
        Inicializa el pipeline con parámetros esenciales.

        :param file_path: Ruta del archivo CSV con datos procesados previos.
        :param output_dir: Directorio donde se guardarán los modelos de clusterización.
        :param num_clusters: Número de clusters a generar con KMeans (por defecto, 5).
        :param random_state: Estado aleatorio para reproducibilidad.
        """
        self.file_path = file_path
        self.output_dir = output_dir
        self.num_clusters = num_clusters
        self.random_state = random_state
        self.data = None
        self.summary = None
        self.kmeans_model = None
        self.scaler = None
        
    def load_data(self):
        """
        Carga el dataset y verifica la presencia de columnas esenciales.
        
        - Convierte columnas numéricas para evitar errores de procesamiento.
        - Elimina valores faltantes en las columnas claves para clustering.
        """
        try:
            self.data = pd.read_csv(self.file_path)
            required_columns = ['Presion', 'Temperatura', 'Volumen', 'Cliente']
            
            if not all(col in self.data.columns for col in required_columns):
                raise ValueError(f"Faltan columnas esenciales: {required_columns}")
            
            # Convertir valores a numérico y eliminar filas con NaN
            for col in ['Presion', 'Temperatura', 'Volumen']:
                self.data[col] = pd.to_numeric(self.data[col], errors='coerce')
                
            self.data.dropna(subset=['Presion', 'Temperatura', 'Volumen'], inplace=True)
            
        
        except FileNotFoundError:
            print(f" ERROR: Archivo {self.file_path} no encontrado.")
            exit()
        except Exception as e:
            print(f" ERROR inesperado: {e}")
            exit()

    def create_client_clusters(self):
        """
        Crea clusters de clientes usando KMeans con datos agregados.

        - Escala los datos antes de aplicar KMeans.
        - Maneja valores faltantes en el resumen mediante imputación.
        - Guarda el modelo de clustering y el escalador usado.
        """
        try:
            self.summary = self.data.groupby("Cliente")[['Presion', 'Temperatura', 'Volumen']].mean()
            self.scaler = StandardScaler()
            summary_scaled = self.scaler.fit_transform(self.summary)

            # Imputar valores faltantes si los hay
            if np.isnan(summary_scaled).any():
                from sklearn.impute import SimpleImputer
                imputer = SimpleImputer(strategy='mean')
                summary_scaled = imputer.fit_transform(summary_scaled)

            self.kmeans_model = KMeans(n_clusters=self.num_clusters, random_state=self.random_state, n_init=10)
            labels = self.kmeans_model.fit_predict(summary_scaled)
            self.summary['cluster'] = labels
            
            # Evaluar calidad del clustering
            silhouette_avg = silhouette_score(summary_scaled, labels)
            print(f" Coeficiente de Silhouette: {silhouette_avg:.3f}")

            # Guardar modelos
            os.makedirs(self.output_dir, exist_ok=True)
            joblib.dump(self.kmeans_model, os.path.join(self.output_dir, 'kmeans_model.joblib'))
            joblib.dump(self.scaler, os.path.join(self.output_dir, 'scaler.joblib'))


        except Exception as e:
            print(f" ERROR en la clusterización: {e}")
            exit()

    def prepare_training_data(self):
        """
        Asocia los clusters de clientes a los registros de consumo y detecta anomalías proxy.

        - Realiza la asociación mediante `merge` con la información de clustering.
        - Calcula anomalías en cada cluster basado en percentiles (P10/P90).
        """
        try:
            summary_reset = self.summary.reset_index()[['Cliente', 'cluster']]
            self.data = self.data.merge(summary_reset, on='Cliente', how='left')

            # Eliminar registros sin cluster asignado
            registros_antes = len(self.data)
            self.data.dropna(subset=['cluster'], inplace=True)
            registros_despues = len(self.data)
            if registros_antes > registros_despues:
                print(f" Se eliminaron {registros_antes - registros_despues} registros sin cluster.")

            # Convertir cluster a entero
            self.data['cluster'] = self.data['cluster'].astype(int)

            print(" Calculando 'Anomalia_Proxy_Cluster' dentro de cada cluster...")
            self.data['Anomalia_Proxy_Cluster'] = 0  # Inicializar columna

            # Iterar sobre cada cluster y calcular anomalía proxy
            for cluster_id in range(self.num_clusters):
                cluster_mask = self.data['cluster'] == cluster_id
                cluster_data = self.data.loc[cluster_mask, ['Presion', 'Temperatura', 'Volumen']]
                
                if cluster_data.empty:
                    continue  # Si el cluster no tiene datos, omitir

                # Calcular percentiles
                anomalies = pd.DataFrame(index=cluster_data.index)
                for feature in ['Presion', 'Temperatura', 'Volumen']:
                    p10, p90 = cluster_data[feature].quantile([0.10, 0.90])
                    anomalies[feature] = cluster_data[feature].apply(lambda x: 1 if x < p10 or x > p90 else 0)

                # Si cualquiera de las tres variables es anómala, marcar como anomalía
                self.data.loc[cluster_mask, 'Anomalia_Proxy_Cluster'] = anomalies.max(axis=1)

        except Exception as e:
            print(f" ERROR en la preparación de entrenamiento: {e}")
            exit()

    def save_training_data(self, output_path="salida/training_data.csv"):
        """
        Guarda los datos preparados en un archivo CSV listo para modelado.

        :param output_path: Ruta del archivo de salida (por defecto, 'training_data.csv').
        """
        self.data.to_csv(output_path, index=False)

In [3]:
#Ejecutando el Pipeline
pipeline = ClusterPipeline("salida/data_cleaned.csv", "modelos_kmeans5_dinamico_otimizado_viz_otim_sem_hist")
pipeline.load_data()
pipeline.create_client_clusters()
pipeline.prepare_training_data()
pipeline.save_training_data()

 Coeficiente de Silhouette: 0.690
 Calculando 'Anomalia_Proxy_Cluster' dentro de cada cluster...
